In [1]:
import numpy as np
import cv2
import sys

In [15]:
model = './opencv_face_detector_uint8.pb'
config = './opencv_face_detector.pbtxt.txt'

img = cv2.imread('./ff.jpeg')

face_net = cv2.dnn.readNet(model, config)

if face_net.empty() :
    print('model load failed')
    sys.exit()

# blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), 
                            swapRB = False) # bgr 로 traing 진행 했다

face_net.setInput(blob)

out = face_net.forward()
print(out.shape)

detect = out[0, 0, : , : ]
print(detect.shape)

h, w = img.shape[:2]

for i in range(200) :
    confidence = detect[i, 2]
    
    if confidence > 0.5 :
        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        
        cv2.rectangle(img, (x1, y1) , (x2, y2), (0,0,255) , 2)

        text = 'Face : {} %'.format(round(confidence * 100, 2))

        cv2.putText(img, text, (x1, y1 - 2), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

(1, 1, 200, 7)
(200, 7)


-1

# 동영상을 이용한 face detector

In [18]:
cap = cv2.VideoCapture(0)

if not cap.isOpened() :
    print('video opend failed')
    sys.exit()

model = './opencv_face_detector_uint8.pb'
config = './opencv_face_detector.pbtxt.txt'

face_net = cv2.dnn.readNet(model, config)

if face_net.empty() :
    print('model load failed')
    sys.exit()
    
while True :
    ret, frame = cap.read()
    frame = cv2.flip(frame , 1)
    
    if not ret:
        print('frame read failed')
        break
        
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123), 
                        swapRB = False) # bgr 로 traing 진행 했다
    face_net.setInput(blob)
    out = face_net.forward()
    detect = out[0, 0, : , :]
    
    h, w = frame.shape[:2]
    
    for i in range(200) :
        confidence = detect[i, 2]
        
        if confidence > 0.5 :
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)

            cv2.rectangle(frame, (x1, y1) , (x2, y2), (0,0,255) , 2)

            text = 'Face : {} %'.format(round(confidence * 100, 2))

            cv2.putText(frame, text, (x1, y1 - 2), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)

        
    cv2.imshow('face', frame)
    
    key = cv2.waitKey(30)
    if key == 27 :
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# caffe 모델 사용

In [21]:
model = './res10_300x300_ssd_iter_140000_fp16.caffemodel'
config = './deploy.prototxt.txt'

net = cv2.dnn.readNet(model, config)

if net.empty() :
    print('model load failed')
    sys.exit()
    
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Go home')
    sys.exit()
    
while True :
    
    ret , frame = cap.read()
    
    if not ret :
        print('frame read failed')
        break
        
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123), swapRB = False)
    net.setInput(blob)
    out = net.forward()
    
    detect = out[0,0, :, :]
    
    h, w = frame.shape[:2]
    
    for i in range(200) :
        confidence = detect[i, 2]
        
        if confidence > 0.5 :
            x1 = int(detect[i,3] * w)
            y1 = int(detect[i,4] * h)
            x2 = int(detect[i,5] * w)
            y2 = int(detect[i,6] * h)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0 , 255), 2)
            text = f'Face : {confidence * 100 :.2f} %'
            
            cv2.putText(frame, text, (x1, y1 - 2), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
        
    cv2.imshow('image', frame)
    
    if cv2.waitKey(20) == 27 :
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1